In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import pandas as pd
from tqdm import tqdm

# Problem 1

In [ ]:
observation_data = pd.read_csv("./seOMXlogreturns2012to2014.csv")
T = observation_data.shape[0]
observation_data = observation_data.to_numpy()[:, 0]
observation_data.shape

In [ ]:
phi = 0.98
sigma = 0.16
betas = np.linspace(0.1, 2, 5)

N = 500

In [ ]:

# Bootstrap Particle Filter
loglikelihood = []

for beta in tqdm(betas):
    loglikelihood_ = []

    for repeat in range(10):

        loglikelihood__ = 0
        initial_particle_dist = scipy.stats.norm(0, 1)
        weights = [np.array([1/N] * N)] + [None] * T
        particles = [initial_particle_dist.rvs(N)] + [None] * T  # draw initial particles
        mean_observation = [None] * T
        prediction = [None] * T
        marginal_filtering = [None] * T

        for t in range(T):
            # RESAMPLE
            ancestor_indices = np.random.choice(range(N), p=weights[t], replace=True, size=N)

            # PROPAGATE
            # state
            proposal_dist = scipy.stats.norm(phi * particles[t][ancestor_indices], sigma)
            particles[t+1] = proposal_dist.rvs()

            # measurement
            measurement_dist = scipy.stats.norm(0, np.sqrt(beta ** 2 * np.exp(particles[t+1])))
            # mean observation
            mean_observation[t] = scipy.stats.norm(0, np.sqrt(beta ** 2 * np.exp(np.mean(particles[t+1])))).rvs()

            # WEIGHT
            log_weights_unnorm = measurement_dist.logpdf(observation_data[t])
            weights_unnorm = np.exp(log_weights_unnorm - np.max(log_weights_unnorm))
            weights[t+1] = weights_unnorm / np.sum(weights_unnorm)

            prediction[t] = np.mean(particles[t])
            marginal_filtering[t] = np.sum(weights[t] * particles[t])

            loglikelihood__ += np.log(np.sum(weights_unnorm)) - np.log(N) + np.max(log_weights_unnorm)

        loglikelihood_.append(loglikelihood__)
    
    loglikelihood.append(loglikelihood_)

    weights = np.array(weights[:-1])
    particles = np.array(particles[:-1])
    mean_observation = np.array(mean_observation)
    prediction = np.array(prediction)
    marginal_filtering = np.array(marginal_filtering)

loglikelihood = np.array(loglikelihood)

In [ ]:
plt.boxplot(loglikelihood.T, positions=betas);

In [ ]:
idx = np.argmax(np.max(loglikelihood, axis=1))
best_beta = betas[idx]

plt.scatter(best_beta, np.max(loglikelihood, axis=1)[idx], c='r', label=f"Optimal beta = {best_beta}")
plt.plot(betas, np.mean(loglikelihood, axis=1));

In [ ]:
plt.scatter(best_beta, np.max(loglikelihood, axis=1)[idx], c='r', label=f"Optimal beta = {best_beta}")
plt.plot(betas, loglikelihood);

Increasing $N$ yields lower variance.

Inreasing $T$ yields lower variance since the influence of the likelihood of the first few timesteps is artificially bad due to the filter not having converged. Increasing $T$ reduces the influence of these first timesteps.

# Problem 2

(i) This is not possible because the observation noise is not Gaussian which makes the $p(y_t|x_t)$ not **conjugate** to $p(x_t|x_{t-1})$ (required for Fully Adapted Particle Filter). We could use a Partially Adapted Particle Filter instead with an approximation of the two.

(ii) This is a Gaussian model and hence OK to implement.

(iii) Cannot implement fully adapted filter since the noise is added inside the cosine.

Fully adapted particle filter

In [ ]:
# Fully Adapted Particle Filter

N = 500

loglikelihood = 0

initial_particle_dist = scipy.stats.norm(1, 1)  # the actual best initial distribution
weights = [np.array([1/N] * N)] + [None] * T  # these are nu weights
particles = [None] * T + [initial_particle_dist.rvs(N)]  # draw initial particles - put at index -1
mean_observation = [None] * T  # p(y_t|x_t)
std_observation = [None] * T
mean_state_prediction = [None] * T  # p(x_t|x_t-1)
std_state_prediction = [None] * T
mean_marginal_filtering = [None] * T  # p(x_t|x_t-1, y_t)
std_marginal_filtering = [None] * T

for t in tqdm(range(T)):
    # RESAMPLE
    # measurement
    fcn = np.cos(particles[t-1]) ** 2
    mean = C * fcn
    sigma = np.sqrt(C * Q * C + R)
    measurement_proposal = scipy.stats.norm(mean, sigma)

    # compute weights (nu)
    log_weights_unnorm = measurement_proposal.logpdf(y_data[t])
    log_weights_max = np.max(log_weights_unnorm)
    weights_unnorm = np.exp(log_weights_unnorm - log_weights_max) + log_weights_max
    weights[t] = weights_unnorm / np.sum(weights_unnorm)

    ancestor_indices = np.random.choice(range(N), p=weights[t], replace=True, size=N)

    # PROPAGATE
    # state
    fcn = np.cos(particles[t-1][ancestor_indices]) ** 2
    mean = fcn + K * (y_data[t] - C * fcn)
    proposal_dist = scipy.stats.norm(mean, np.sqrt(Sigma))
    particles[t] = proposal_dist.rvs()
    # measurement (optional)
    measurement_dist = scipy.stats.norm(C * np.mean(particles[t]), np.sqrt(R))
    mean_observation[t] = measurement_dist.mean()
    std_observation[t] = measurement_dist.std()

    # mean_marginal_filtering[t] = np.mean(proposal_dist.mean())  # particles incorporate y_data from same time step (hence filtering)
    mean_marginal_filtering[t] = np.mean(proposal_dist.rvs())  # particles incorporate y_data from same time step (hence filtering)
    std_marginal_filtering[t] = np.mean(proposal_dist.std())

    fcn = np.cos(particles[t-1]) ** 2  # no resampling here
    prediction_dist = scipy.stats.norm(fcn, np.sqrt(Q))  # prediction formed by ignoring y_data (not available)
    mean_state_prediction[t] = np.mean(prediction_dist.mean())
    std_state_prediction[t] = np.mean(prediction_dist.std())

    # loglikelihood += np.log(np.sum(weights_unnorm)) - np.log(N)

weights = np.array(weights[:-1])
particles = np.array(particles[-1:] + particles[1:-1])  # move initial particle to index 0  #  np.array(particles[:-1])
mean_marginal_filtering = np.array(mean_marginal_filtering)
std_marginal_filtering = np.array(std_marginal_filtering)
mean_state_prediction = np.array(mean_state_prediction)
std_state_prediction = np.array(std_state_prediction)
mean_observation = np.array(mean_observation)
std_observation = np.array(std_observation)
loglikelihood = np.array(loglikelihood)

In [ ]:
plt.plot(mean_marginal_filtering)
plt.plot(std_marginal_filtering)

In [ ]:
plt.plot(mean_marginal_filtering)
plt.plot(mean_state_prediction);

In [ ]:
plt.plot(y_data)
plt.plot(mean_observation)
plt.plot(std_observation);

In [ ]:
plt.plot(y_data[100:200])
plt.plot(mean_observation[100:200])

In [ ]:
plt.plot(y_data[100:102])
plt.plot(mean_observation[100:102])

### Comparison of APF and BPF estimator variance

In [ ]:
# Bootstrap Particle Filter

bpf_means_of_estimates = []

for repeat in range(10):

    initial_particle_dist = scipy.stats.norm(1, 1)
    weights = [np.array([1/N] * N)] + [None] * T
    particles = [initial_particle_dist.rvs(N)] + [None] * T  # draw initial particles
    mean_observation = [None] * T
    prediction = [None] * T
    marginal_filtering = [None] * T

    for t in tqdm(range(T)):
        # RESAMPLE
        ancestor_indices = np.random.choice(range(N), p=weights[t], replace=True, size=N)

        # PROPAGATE
        # state
        fcn = np.cos(particles[t][ancestor_indices]) ** 2
        proposal_dist = scipy.stats.norm(fcn, np.sqrt(Q))
        particles[t+1] = proposal_dist.rvs()

        # measurement
        measurement_dist = scipy.stats.norm(2 * particles[t+1], np.sqrt(R))
        # mean observation
        mean_observation[t] = scipy.stats.norm(2 * np.mean(particles[t+1]), np.sqrt(R)).rvs()

        # WEIGHT
        weights[t+1] = measurement_dist.logpdf(y_data[t])
        weights[t+1] = np.exp(weights[t+1] - np.max(weights[t+1]))
        weights[t+1] = weights[t+1] / np.sum(weights[t+1])

        prediction[t] = np.mean(particles[t])
        marginal_filtering[t] = np.sum(weights[t] * particles[t])

    weights = np.array(weights[:-1])
    particles = np.array(particles[:-1])
    mean_observation = np.array(mean_observation)
    prediction = np.array(prediction)
    marginal_filtering = np.array(marginal_filtering)

    bpf_means_of_estimates.append(marginal_filtering)
    
bpf_means_of_estimates = np.array(bpf_means_of_estimates)

In [ ]:
# Fully Adapted Particle Filter

N = 500

apf_means_of_estimates = []

for repeat in range(10):
    loglikelihood = 0

    initial_particle_dist = scipy.stats.norm(1, 1)  # the actual best initial distribution
    weights = [np.array([1/N] * N)] + [None] * T  # these are nu weights
    particles = [None] * T + [initial_particle_dist.rvs(N)]  # draw initial particles - put at index -1
    mean_observation = [None] * T  # p(y_t|x_t)
    std_observation = [None] * T
    # mean_observation_prediction = [None] * T  # p(y_t|x_t-1)
    # std_observation_prediction = [None] * T
    mean_state_prediction = [None] * T  # p(x_t|x_t-1)
    std_state_prediction = [None] * T
    mean_marginal_filtering = [None] * T  # p(x_t|x_t-1, y_t)
    std_marginal_filtering = [None] * T

    for t in tqdm(range(T)):
        # RESAMPLE
        # measurement
        fcn = np.cos(particles[t-1]) ** 2
        mean = C * fcn
        sigma = np.sqrt(C * Q * C + R)
        measurement_proposal = scipy.stats.norm(mean, sigma)

        # compute weights (nu)
        log_weights_unnorm = measurement_proposal.logpdf(y_data[t])
        log_weights_max = np.max(log_weights_unnorm)
        weights_unnorm = np.exp(log_weights_unnorm - log_weights_max) + log_weights_max
        weights[t] = weights_unnorm / np.sum(weights_unnorm)

        ancestor_indices = np.random.choice(range(N), p=weights[t], replace=True, size=N)

        # PROPAGATE
        # state
        fcn = np.cos(particles[t-1][ancestor_indices]) ** 2
        mean = fcn + K * (y_data[t] - C * fcn)
        proposal_dist = scipy.stats.norm(mean, np.sqrt(Sigma))
        particles[t] = proposal_dist.rvs()
        # measurement (optional)
        measurement_dist = scipy.stats.norm(C * np.mean(particles[t]), np.sqrt(R))
        mean_observation[t] = measurement_dist.mean()
        std_observation[t] = measurement_dist.std()

        # mean_marginal_filtering[t] = np.mean(proposal_dist.mean())  # particles incorporate y_data from same time step (hence filtering)
        mean_marginal_filtering[t] = np.mean(proposal_dist.rvs())  # particles incorporate y_data from same time step (hence filtering)
        std_marginal_filtering[t] = np.mean(proposal_dist.std())

        fcn = np.cos(particles[t-1]) ** 2  # no resampling here
        prediction_dist = scipy.stats.norm(fcn, np.sqrt(Q))  # prediction formed by ignoring y_data (not available)
        mean_state_prediction[t] = np.mean(prediction_dist.mean())
        std_state_prediction[t] = np.mean(prediction_dist.std())

        # loglikelihood += np.log(np.sum(weights_unnorm)) - np.log(N)

    weights = np.array(weights[:-1])
    particles = np.array(particles[-1:] + particles[1:-1])  # move initial particle to index 0  #  np.array(particles[:-1])
    mean_marginal_filtering = np.array(mean_marginal_filtering)
    std_marginal_filtering = np.array(std_marginal_filtering)
    mean_state_prediction = np.array(mean_state_prediction)
    std_state_prediction = np.array(std_state_prediction)
    mean_observation = np.array(mean_observation)
    std_observation = np.array(std_observation)
    loglikelihood = np.array(loglikelihood)

    apf_means_of_estimates.append(mean_marginal_filtering)
    
apf_means_of_estimates = np.array(apf_means_of_estimates)

In [ ]:
mean_of_estimate.shape

In [ ]:
plt.plot(np.std(bpf_means_of_estimates, axis=0), label="BPF variance")
plt.plot(np.std(apf_means_of_estimates, axis=0), label="APF variance")
plt.legend()
plt.xlabel("t")
plt.ylabel("Estimator variance")
plt.yscale("log")
print(np.mean(np.std(bpf_means_of_estimates, axis=0)))
print(np.mean(np.std(apf_means_of_estimates, axis=0)))

# Problem 3

# Problem 4